In [ ]:
import cv2
import imageio
import numpy as np
import plotly.express as px
from skimage.morphology import skeletonize

def load_image(idx=0):
    vid = imageio.get_reader("Ludosport_birthday_sparring.mp4",  'ffmpeg')
    vid_iter = vid.iter_data()
    for _ in range(idx+1):
        image = next(vid_iter)
    return image

def variance_of_laplacian(image, thresh=100):
    # https://pyimagesearch.com/2015/09/07/blur-detection-with-opencv/
    # compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    variance = cv2.Laplacian(image, cv2.CV_64F).var()
    text = "Not Blurry"
    if variance < thresh:
        text = "Blurry"
    return f"{text}: {variance:.2f}"

img = load_image(10)

blur_text = variance_of_laplacian(img, thresh=300)
cv2.putText(img, blur_text, (10, 50),
            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 2)
px.imshow(img)

In [ ]:
# reload image to ensure reset when running cell
test_frames = (51, 56, 78, 104, 174, 454)
idx = test_frames[-1]
img = load_image(idx)

# estimate blur in image
blur_text = variance_of_laplacian(img, thresh=300)

# convert to HSV for simplicity
blur = cv2.GaussianBlur(img, (5, 5), 0)
m1 = blur[:, :, 1] > 200
m2 = blur[:, :, 2] > 200
m3 = blur[:, :, 0] > 240
mask = (m1 + m2 + m3).astype(np.uint8)

rho = 1                         # distance resolution in pixels of the Hough grid
theta = np.pi / 180             # angular resolution in radians of the Hough grid
threshold = 40                  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 25            # minimum number of pixels making up a line
max_line_gap = 10               # maximum gap in pixels between connectable line segments

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
lines = cv2.HoughLinesP(mask, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)

gray = np.zeros(img.shape[:2], np.uint8)
if isinstance(lines, np.ndarray):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(gray, (x1, y1), (x2, y2), 255, 2)

contours, _ = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
for i, contour in enumerate(contours):
    if 800 > cv2.contourArea(contour) > 60:
        cv2.drawContours(img, contours, i, 255, -1)

cv2.putText(img, blur_text, (10, 50),
            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 2)
px.imshow(img)

In [ ]:
np.array([cv2.contourArea(i) for i in contours], dtype=np.uint16) # contour areas

In [ ]:
# comb through RGB channels to find best combination
mask = blur[:, :, 2] > 200
px.imshow(mask)